# Phase 3: Feature Extraction (Croce et al. 2021)
## Alaca Cesmesi Scan-to-HBIM V6 Pipeline

Extracts 30 geometric features:
- 9 eigenvalue-based features x 3 scales = 27 features
- 3 position features: X, **Y**, Z normalized

**CRITICAL: Y_normalized is the key feature for 99.86% accuracy!**

**Input:** `gs://alaca-cesme-hbim-v6/processed/v{N}/02_preprocessed/`  
**Output:** `gs://alaca-cesme-hbim-v6/processed/v{N}/03_features/`

In [ ]:
!pip install -q open3d google-cloud-storage numpy scipy

In [ ]:
import open3d as o3d
import numpy as np
from scipy.spatial import cKDTree
import json
import os
import time
from datetime import datetime
from google.cloud import storage
from google.colab import auth

auth.authenticate_user()

# Configuration
BUCKET_NAME = "alaca-cesme-hbim-v6"
PROJECT_ID = "concrete-racer-470219-h8"
VERSION = "v1"

# Feature extraction parameters (Croce et al. 2021)
RADII = [0.03, 0.06, 0.10]  # Multi-scale radii
FEATURE_NAMES = [
    "linearity", "planarity", "sphericity", "omnivariance",
    "eigenentropy", "surface_variation", "sum_eigenvalues",
    "anisotropy", "verticality"
]

# Paths
INPUT_PATH = f"processed/{VERSION}/02_preprocessed/02_preprocessed.ply"
OUTPUT_BASE = f"processed/{VERSION}/03_features/"

LOCAL_INPUT = "/content/input.ply"
LOCAL_OUTPUT_NPY = "/content/03_features.npy"
LOCAL_OUTPUT_JSON = "/content/03_features_stats.json"

In [ ]:
# GCS functions
def download_from_gcs(bucket_name, blob_name, local_path):
    client = storage.Client(project=PROJECT_ID)
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.download_to_filename(local_path)
    print(f"Downloaded: {blob_name}")
    return local_path

def upload_to_gcs(bucket_name, local_path, blob_name):
    client = storage.Client(project=PROJECT_ID)
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(local_path)
    print(f"Uploaded: {blob_name}")
    return f"gs://{bucket_name}/{blob_name}"

# Download input
download_from_gcs(BUCKET_NAME, INPUT_PATH, LOCAL_INPUT)
pcd = o3d.io.read_point_cloud(LOCAL_INPUT)
points = np.asarray(pcd.points)
normals = np.asarray(pcd.normals)
n_points = len(points)
print(f"Loaded: {n_points:,} points")

In [ ]:
def compute_eigenvalues(points, kdtree, radius, min_neighbors=5):
    """
    Compute eigenvalues for each point using PCA of local neighborhood.
    """
    n_points = len(points)
    eigenvalues = np.zeros((n_points, 3))
    eps = 1e-10
    
    for i in range(n_points):
        indices = kdtree.query_ball_point(points[i], radius)
        
        if len(indices) < min_neighbors:
            eigenvalues[i] = [1.0, 1.0, 1.0]
            continue
        
        neighbors = points[indices]
        centered = neighbors - neighbors.mean(axis=0)
        cov = np.cov(centered.T)
        
        if cov.ndim < 2:
            eigenvalues[i] = [1.0, 0.0, 0.0]
            continue
        
        eigvals = np.linalg.eigvalsh(cov)
        eigvals = np.sort(eigvals)[::-1]  # Descending
        eigvals = np.maximum(eigvals, eps)
        eigenvalues[i] = eigvals
    
    return eigenvalues


def compute_geometric_features(eigenvalues, normals):
    """
    Compute 9 geometric features from eigenvalues and normals.
    """
    n_points = len(eigenvalues)
    features = np.zeros((n_points, 9))
    eps = 1e-10
    
    l1, l2, l3 = eigenvalues[:, 0], eigenvalues[:, 1], eigenvalues[:, 2]
    l_sum = l1 + l2 + l3 + eps
    
    # 9 features (Croce et al. 2021)
    features[:, 0] = (l1 - l2) / (l1 + eps)  # Linearity
    features[:, 1] = (l2 - l3) / (l1 + eps)  # Planarity
    features[:, 2] = l3 / (l1 + eps)          # Sphericity
    features[:, 3] = np.cbrt(l1 * l2 * l3 + eps)  # Omnivariance
    
    l_norm = eigenvalues / l_sum[:, np.newaxis]
    l_norm = np.maximum(l_norm, eps)
    features[:, 4] = -np.sum(l_norm * np.log(l_norm), axis=1)  # Eigenentropy
    
    features[:, 5] = l3 / l_sum  # Surface variation
    features[:, 6] = l_sum       # Sum eigenvalues
    features[:, 7] = (l1 - l3) / (l1 + eps)  # Anisotropy
    features[:, 8] = 1.0 - np.abs(normals[:, 2])  # Verticality
    
    return features

In [ ]:
start_time = time.time()

# Build KD-tree once
print("Building KD-tree...")
kdtree = cKDTree(points)

# Extract features at each scale
n_features_per_scale = 9
all_features = np.zeros((n_points, n_features_per_scale * len(RADII)))

for scale_idx, radius in enumerate(RADII):
    print(f"\n[Scale {scale_idx + 1}/{len(RADII)}] radius = {radius}m")
    
    eigenvalues = compute_eigenvalues(points, kdtree, radius)
    features = compute_geometric_features(eigenvalues, normals)
    
    start_col = scale_idx * n_features_per_scale
    end_col = start_col + n_features_per_scale
    all_features[:, start_col:end_col] = features
    
    print(f"  Computed {n_features_per_scale} features")

print(f"\nGeometric features shape: {all_features.shape}")

In [ ]:
# Add X, Y, Z normalized - CRITICAL FOR ACCURACY!
print("\nAdding normalized position features (X, Y, Z)...")

# X normalized
x_coords = points[:, 0]
x_normalized = (x_coords - x_coords.min()) / (x_coords.max() - x_coords.min() + 1e-10)

# Y normalized - THE KEY FEATURE FOR 99.86% ACCURACY!
y_coords = points[:, 1]
y_normalized = (y_coords - y_coords.min()) / (y_coords.max() - y_coords.min() + 1e-10)

# Z normalized
z_coords = points[:, 2]
z_normalized = (z_coords - z_coords.min()) / (z_coords.max() - z_coords.min() + 1e-10)

# Combine all features
features = np.column_stack([all_features, x_normalized, y_normalized, z_normalized])

elapsed_time = time.time() - start_time
print(f"\nFinal feature shape: {features.shape}")
print(f"Total features: {features.shape[1]} (27 geometric + 3 position)")
print(f"Processing time: {elapsed_time:.1f} seconds")

In [ ]:
# Create feature names
feature_names = []
for radius in RADII:
    for name in FEATURE_NAMES:
        feature_names.append(f"{name}_r{radius:.2f}")
feature_names.extend(["x_normalized", "y_normalized", "z_normalized"])

# Compute statistics
feature_stats = {}
for i, name in enumerate(feature_names):
    col = features[:, i]
    feature_stats[name] = {
        "min": float(np.min(col)),
        "max": float(np.max(col)),
        "mean": float(np.mean(col)),
        "std": float(np.std(col))
    }

stats = {
    "phase": "03_features",
    "n_points": n_points,
    "n_features": features.shape[1],
    "feature_names": feature_names,
    "radii": RADII,
    "y_normalized_note": "CRITICAL: Y_normalized is the key feature for 99.86% accuracy",
    "feature_statistics": feature_stats,
    "processing_time_sec": elapsed_time,
    "timestamp": datetime.now().isoformat(),
    "pipeline_version": "v6"
}

# Save locally
np.save(LOCAL_OUTPUT_NPY, features)
with open(LOCAL_OUTPUT_JSON, 'w') as f:
    json.dump(stats, f, indent=2)
print("Saved outputs locally")

In [ ]:
# Upload to GCS
upload_to_gcs(BUCKET_NAME, LOCAL_OUTPUT_NPY, f"{OUTPUT_BASE}03_features.npy")
upload_to_gcs(BUCKET_NAME, LOCAL_OUTPUT_JSON, f"{OUTPUT_BASE}03_features_stats.json")

# Status for n8n
status = {
    "phase": "03_features",
    "status": "success",
    "version": VERSION,
    "outputs": {
        "features": f"gs://{BUCKET_NAME}/{OUTPUT_BASE}03_features.npy",
        "stats": f"gs://{BUCKET_NAME}/{OUTPUT_BASE}03_features_stats.json"
    },
    "metrics": {
        "n_points": n_points,
        "n_features": features.shape[1],
        "radii": RADII,
        "processing_time": f"{elapsed_time:.1f}s"
    },
    "timestamp": datetime.now().isoformat(),
    "next_phase": "04_segment"
}

print("\n" + "="*60)
print("PHASE 3 COMPLETE")
print("="*60)
print(json.dumps(status, indent=2))